In [1]:
'''采用自组CNN模型进行猫狗图片二分类'''
import os, shutil
from keras import layers
from keras import models
from keras import optimizers
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# 原始数据集的路径
original_dataset_dir = '/home/Howie/alldataset/catsdogsdataset'

# 项目数据集存放路径
base_dir = '/home/Howie/kerasproject/cats_dogs/dataset'

# 创建训练集、验证集、测试集目录
train_dir = os.path.join(base_dir, 'train')
##os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
##os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
##os.mkdir(test_dir)

# 创建猫狗目录
train_cats_dir = os.path.join(train_dir, 'cats')
validation_cats_dir = os.path.join(validation_dir, 'cats')
test_cats_dir = os.path.join(test_dir, 'cats')
##os.mkdir(train_cats_dir)
##os.mkdir(validation_cats_dir)
##os.mkdir(test_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_dogs_dir = os.path.join(test_dir, 'dogs')
##os.mkdir(train_dogs_dir)
##os.mkdir(validation_dogs_dir)
##os.mkdir(test_dogs_dir)

#数据集复制
fnames = ['cat.{}.jpg'.format(i) for i in range(3000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(scr, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(3000, 4000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(scr, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(4000, 5000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(scr, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(3000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(scr, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(3000, 4000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(scr, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(4000, 5000)]
for fname in fnames:
    scr = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(scr, dst)

In [3]:
# 建立CNN模型
def generate_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu',
    input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    # model.summary()
    return model


In [4]:
# 图像生成器
train_dategen = ImageDataGenerator(rescale=1./255)
validation_dategen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,)

train_generator = train_dategen.flow_from_directory(train_dir, 
 target_size=(150,150), batch_size=256, class_mode='binary')
validation_generator = validation_dategen.flow_from_directory(validation_dir,
 target_size=(150,150), batch_size=256, class_mode='binary')

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
data batch shape: (256, 150, 150, 3)
labels batch shape: (256,)


In [5]:

model = generate_model() 
acc, val_acc, loss, val_loss = [], [], [], []
for i in range(10):
    print("Learning with rate of {}".format(1e-3/(1.8**i)))
    model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-3/(1.8**i)),
    loss='binary_crossentropy', metrics=['acc'])
    # 训练模型
    history_bin = model.fit_generator(train_generator, 
    epochs=10, validation_data=validation_generator,)

    
    acc = acc + history_bin.history['acc'] 
    val_acc = val_acc + history_bin.history['val_acc'] 
    loss = loss + history_bin.history['loss']
    val_loss = val_loss + history_bin.history['val_loss'] 
    
    #loss.append( history_bin.history['loss'][-1] )


#plt.plot(range(len(lrs)), loss)
# 保存模型
model.save('/home/Howie/kerasproject/cats_dogs/cats_and_dogs_1.h5')# 训练模型


Learning with rate of 0.001
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
24/24 [==============================] - 35s 1s/step - loss: 0.8837 - acc: 0.5123 - val_loss: 0.6904 - val_acc: 0.5010
Epoch 2/10
24/24 [==============================] - 35s 1s/step - loss: 0.6918 - acc: 0.5368 - val_loss: 0.6841 - val_acc: 0.6075
Epoch 3/10
24/24 [==============================] - 35s 1s/step - loss: 0.6909 - acc: 0.5475 - val_loss: 0.6807 - val_acc: 0.5790
Epoch 4/10
24/24 [==============================] - 35s 1s/step - loss: 0.6809 - acc: 0.5713 - val_loss: 0.6584 - val_acc: 0.5990
Epoch 5/10
24/24 [==============================] - 35s 1s/step - loss: 0.6563 - acc: 0.6198 - val_loss: 0.6482 - val_acc: 0.6220
Epoch 6/10
24/24 [==============================] - 35s 1s/step - loss: 0.6655 - acc: 0.6287 - val_loss: 0.6084 - val_acc: 0.6795
Epoch 7/10
24/24 [==============================] - 35s 1s/step - loss: 0.6387 - acc: 0.6457 - val_loss: 0.5935 - va

KeyboardInterrupt: 

In [4]:
# 训练过程可视化
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(1, len(acc)+1)


plt.plot(epoch, acc, 'b', label='Training accuracy')
plt.plot(epoch, val_acc, 'y', label='Validation_accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')

plt.legend()

NameError: name 'history' is not defined